## Loading the cleaned data

In [1]:
# File path: Amy
import pandas as pd 
import os
import zipfile
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
file_path = os.path.join(parent_directory, "MasterBayesianBookRec/archive/Combined.csv")
df = pd.read_csv(file_path)

/var/folders/zr/hq5f8g3s0jq6yt_pl2ppbgmc0000gn/T/ipykernel_35410/2517243579.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [2]:
df.head()

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Location,Age,Title_Author
0,0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN,Flesh Tones: A Novel || M. J. Rose
1,1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,"seattle, washington, usa",NaN,Rites of Passage || Judith Rae
2,2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,"h, new south wales, australia",16.0,The Notebook || Nicholas Sparks
3,3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia",16.0,Help!: Level 1 || Philip Prowse
4,4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031134 entries, 0 to 1031133
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031134 non-null  int64  
 1   User-ID              1031134 non-null  int64  
 2   ISBN                 1031134 non-null  object 
 3   Book-Rating          1031134 non-null  int64  
 4   Book-Title           1031134 non-null  object 
 5   Book-Author          1031134 non-null  object 
 6   Year-Of-Publication  1031134 non-null  object 
 7   Publisher            1031132 non-null  object 
 8   Image-URL-S          1031134 non-null  object 
 9   Location             1031134 non-null  object 
 10  Age                  753299 non-null   float64
 11  Title_Author         1031134 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 94.4+ MB


In [9]:
import numpy as np

# Select relevant columns
df = df[['User-ID', 'ISBN', 'Book-Rating']]

# Encode User-ID and ISBN as categorical
df['User-Index'] = df['User-ID'].astype("category").cat.codes
df['Book-Index'] = df['ISBN'].astype("category").cat.codes

# Normalize ratings to be between 0 and 1 for Bayesian modeling
df['Rating-Norm'] = df['Book-Rating'] / df['Book-Rating'].max()

# Get number of unique users and books
num_users = df['User-Index'].nunique()
num_books = df['Book-Index'].nunique()

# Convert to numpy arrays for modeling
user_ids = df['User-Index'].values
book_ids = df['Book-Index'].values
ratings = df['Rating-Norm'].values

print("Number of unique users:", num_users)
print("Number of unique books:", num_books)
df.head()

Number of unique users: 92106
Number of unique books: 270149


,User-ID,ISBN,Book-Rating,User-Index,Book-Index,Rating-Norm
0,276725,034545104X,0,91362,45921,0.0
1,276726,0155061224,5,91363,22731,0.5
2,276727,0446520802,0,91364,92659,0.0
3,276729,052165615X,3,91365,111478,0.3
4,276729,0521795028,6,91365,111512,0.6


In [14]:
# Install PyMC3 for MCMC modeling
pip install pymc3

SyntaxError: invalid syntax (1658139351.py, line 2)

In [2]:
pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
# Set latent dimension for user and book embeddings
latent_dim = 3  

# Bayesian Probabilistic Matrix Factorization Model
with pm.Model() as model:
    # Global mean prior
    mu = pm.Normal("mu", mu=0, sigma=1)

    # User & book bias
    user_bias = pm.Normal("user_bias", mu=0, sigma=1, shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=1, shape=num_books)

    # User and book latent factors
    user_factors = pm.Normal("user_factors", mu=0, sigma=1, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=1, shape=(num_books, latent_dim))

    # Noise term
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected rating (likelihood function)
    rating_mean = (
        mu +
        user_bias[user_ids] +
        book_bias[book_ids] +
        (user_factors[user_ids] * book_factors[book_ids]).sum(axis=1)
    )

    # Likelihood function for observed ratings
    ratings_obs = pm.Normal("ratings_obs", mu=rating_mean, sigma=sigma, observed=ratings)

    # Run MCMC Sampling using NUTS (No-U-Turn Sampler)
    trace = pm.sample(500, tune=500, target_accept=0.9, cores=2)

    # Plot Posterior Distributions
pm.traceplot(trace)
plt.show()

# Predict missing ratings using posterior means
posterior_means = (
    trace["mu"].mean() +
    trace["user_bias"].mean(axis=0)[:, None] +
    trace["book_bias"].mean(axis=0)[None, :] +
    np.dot(trace["user_factors"].mean(axis=0), trace["book_factors"].mean(axis=0).T)
)

# Print example of predicted ratings
print("\nExample of Predicted Ratings (posterior mean):")
print(posterior_means[:5, :5])  # Print a small part of the matrix


ModuleNotFoundError: No module named 'distutils.msvccompiler'

In [4]:
pip install setuptools


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymc


  Using cached arviz-0.20.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 28.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for cons: filename=cons-0.4.6-py3-none-any.whl size=9094 sha256=2e808e716a7d993705b38897dec9c0b2f4f09628382f98f6f4986defa382b7de
  Stored in directory: /Users/hyunjikim/Library/Caches/pip/wheels/a4/11/f5/8265470e17d280d8b5910e1b0f298521420a5086d696e96799
  Created wheel for logical-unification: filename=logical_unification-0.4.6-py3-none-any.whl size=13912 sha256=